# WEEK 19 (05/03/18)

Time to prepare all the key plots in a standardised way (same label size, style, colours, etc.) and save them in high resolution.

This notebook is a collection of code snippets to plot a few graphs containing results and some visualisations.

## Visualisations

In [81]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas 
from matplotlib import pyplot as plt
import seaborn as sns

from hm.pop_models.pop_explicit import explicit as pop_explicit
from hm.analysis import random_tripoint as rt
from hm.pop_models.pop_random import random as pop_random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Random Tripoint

In [247]:
p = pop_random(100)

In [248]:
x = p.locCoords[:,0]
y = p.locCoords[:,1]

b = np.random.choice(range(len(rt.neighbours(p)[1])))
j, k = rt.neighbours(p)[1][b]

while rt.neighbours(p)[0][j][1] < 0.05:
    b = np.random.choice(range(len(rt.neighbours(p)[1])))
    j, k = rt.neighbours(p)[1][b]
    
i = np.random.choice(range(len(rt.neighbours(p))))
while rt.r_ib(p, i)[j-1] > 0.3:
    i = np.random.choice(range(len(rt.neighbours(p))))

In [249]:
mid_x = (x[j]+ x[k])/2
mid_y = (y[j]+ y[k])/2

In [ ]:
# Global settings
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-deep')

# Resolution
fig = plt.figure(figsize=(800/110.27, 800/110.27), dpi=300)

# Axes/tick labels
plt.xlabel(r'x', fontsize=20)
plt.ylabel(r'y', fontsize=20)
plt.tick_params(axis='both', labelsize=15)

# stops lables being cut off
plt.tight_layout()

plt.scatter(x, y,color= sns.xkcd_rgb["grey"])

#target location i
plt.scatter(x[i], y[i],color= sns.xkcd_rgb["pale red"], s = 150.)
plt.annotate('i', (x[i], y[i]+0.01), size = 20)

#clustered locations j, k
plt.scatter(x[k], y[k], color= sns.xkcd_rgb["medium green"], s = 150.)
plt.annotate('k', (x[k]+0.02, y[k]-0.02), size = 20)
plt.scatter(x[j], y[j], color= sns.xkcd_rgb["medium green"], s = 150.)
plt.annotate('j', (x[j]-0.03, y[j]+0.02), size = 20)

#arrows for r_ib and r_jk
plt.annotate("", xytext=(x[i], y[i]), xy=(mid_x, mid_y),arrowprops=dict(arrowstyle="<->", connectionstyle="arc3"))
plt.annotate("", xy=(x[j], y[j]), xytext=(x[k], y[k]),arrowprops=dict(arrowstyle="<->", connectionstyle="arc3"))

#text for r_ib and r_jk
plt.annotate("$r_{jk}$", (mid_x, mid_y+0.01), size = 20)
plt.annotate("$r_{ib}$", (0.50, 0.55), size = 20)

plt.legend(frameon=False, fontsize=20)
plt.savefig("random tripoint", transparent=True)

Radiation model plot:

In [257]:
# Global settings
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-deep')

# Resolution
fig = plt.figure(figsize=(800/110.27, 800/110.27), dpi=300)

# Axes/tick labels
plt.xlabel(r'x', fontsize=20)
plt.ylabel(r'y', fontsize=20)
plt.tick_params(axis='both', labelsize=15)

# stops lables being cut off
plt.tight_layout()

plt.scatter(x, y,color= sns.xkcd_rgb["grey"])
plt.scatter(x[i], y[i],color= sns.xkcd_rgb["pale red"], s = 150.)
plt.annotate('i', (x[i], y[i]+0.01), size = 20)

plt.scatter(x[j], y[j], color= sns.xkcd_rgb["medium green"], s = 150.)
plt.annotate('j', (x[j]-0.03, y[j]+0.02), size = 20)
plt.annotate("$s_{ij}$", (0.3, 0.6), size = 50)


radius = np.sqrt((x[j]-x[i])**2 + (y[j]-y[i])**2)
circle = plt.Circle((x[i], y[i]), radius, alpha = 0.1, linewidth = 10.)
plt.gcf().gca().add_artist(circle)

plt.savefig("random tripoint", transparent=True)

/Users/Ilaria/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


### Clusters

#### UK

Code to plot the all the UK locations clustered in a nice and clear way:

In [9]:
import fastcluster
from scipy.cluster import hierarchy as hier

from hm.coarse_grain import clustering
import seaborn as sns; sns.set()

In [5]:
df = pandas.read_csv("/Users/Ilaria/Documents/Imperial/MSci_Project/Code/human_mob/data/LSOA populations and areas 2011 (merged).csv")

In [ ]:
df = pandas.read_csv("/Users/Ilaria/Documents/Imperial/MSci_Project/Code/human_mob/data/LSOA populations and areas 2011 (merged).csv")
x = np.array(df)[:,5]
y = np.array(df)[:,6]
m = np.array(df)[:,2]
xy = np.array([x, y])
xy = np.transpose(xy)
#NB: the new version of pop_explicit() calls the pdist() function at initialisation
#this results in a long runtime (and RAM usage) for the whole of the UK dataset
#--> comment out the DM matrix function in the main code to avoid this
p = pop_explicit(xy, m)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('seaborn-deep')
plt.figure(figsize=(900/110.27, 1200/110.27), dpi = 300)
plt.xlabel(r'Eastings (Km)', fontsize=20)
plt.ylabel(r'Northings (Km)', fontsize=20)
plt.tick_params(axis='both', labelsize=15)
plt.ticklabel_format(style='sci')
plt.axis('equal')
xy = p.locCoords
palette = sns.color_palette()
palette = palette*int(len(clus)/6)
colors = []
for i in clus:
    colors.append(palette[i])
plt.scatter(xy[:,0], xy[:,1], s = .8, c = colors)
plt.savefig('UK', transparent=True)

#### London (Voronoi tesselation)

In [1]:
from scipy.spatial import Voronoi, voronoi_plot_2d

In [6]:
lon_df = df[df['Easting'] < 561966][df['Easting'] > 493024][df['Northing'] < 207218][df['Northing'] > 150912]

/Users/Ilaria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [7]:
x_lon = np.array(lon_df)[:,5]
y_lon = np.array(lon_df)[:,6]
m_lon = np.array(lon_df)[:,2]
xy_lon = np.array([x_lon, y_lon])
xy_lon = np.transpose(xy_lon)
p_lon = pop_explicit(xy_lon, m_lon)

In [8]:
lon_2000 = clustering.Clusters(p_lon, 2000)

In [ ]:
lon_200.viz_clusters()